<a href="https://colab.research.google.com/github/KML1103/NLPCW1-IDENTIFYING-AND-CATEGORIZING-OFFENISVE-LANGUAGE-IN-SOCIAL-MEDIA-/blob/main/Xenophobia_on_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [18]:
for dirname, _, filenames in os.walk('/content/sample_data/Xenophobia.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [19]:
#Distribution graphs

def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]]

    # For displaying purposes, pick columns that have between 1 and 50 unique values

    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [20]:
# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

In [21]:
# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [22]:
sample = 10000
# Xenophobia.csv may have more rows in reality, but we are only loading/previewing the first 10000 rows
df1 = pd.read_csv('/content/sample_data/Xenophobia.csv', delimiter=',',nrows=sample,encoding='latin1')
df1.dataframeName = 'Xenophobia.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 10000 rows and 9 columns


In [23]:
df1.head(20)

,status_id,text,created_at,favorite_count,retweet_count,location,followers_count,friends_count,statuses_count
0,1236048892484939776,wash yo hands\nstay updated\nwash em again\nea...,2020-03-06T21:59:55Z,0,0,God's love,493,225,12262
1,1236048892862574594,The President inside the with its leadership...,2020-03-06T21:59:55Z,0,0,"Atlanta, GA",10450,3722,15102
2,1236048899925774337,Itâs hard to focus on anything else with the...,2020-03-06T21:59:57Z,0,0,"Nashville, TN",6113,679,3932
3,1236049533655830528,"I mean, seriously. The Acting Deputy Secretary...",2020-03-06T22:02:28Z,0,0,"Merrimack Valley, Massachusetts, USA https://k...",6384,1593,189120
4,1236048901540585472,"""""Cuccinelli defended the decision, saying the...",2020-03-06T21:59:57Z,0,0,"Merrimack Valley, Massachusetts, USA https://k...",6384,1593,189120
5,1236048910977769472,UPDATE: All the three suspected cases; travel...,2020-03-06T21:59:59Z,0,0,worldwide,311,132,9939
6,1236049503599448064,(1) here is what concerns us....we have been t...,2020-03-06T22:02:21Z,0,0,NaN,0,0,5
7,1236049143342252032,rampant reports of Asian Americans being assau...,2020-03-06T22:00:55Z,0,0,NaN,0,0,5
8,1236048899707674625,"33 cases now in N.Y., with most connected to W...",2020-03-06T21:59:57Z,0,0,NaN,0,0,5
9,1236050100536963074,no way in hell would a trip on a cruise be som...,2020-03-06T22:04:43Z,0,0,NaN,0,0,5


In [24]:
#distribution graphs of sampled columns


plotPerColumnDistribution(df1, 10, 5)
plotPerColumnDistribution


<Figure size 2400x512 with 0 Axes>

<function __main__.plotPerColumnDistribution(df, nGraphShown, nGraphPerRow)>

In [35]:
#reference: https://medium.com/sculpt/xenophobic-tweets-78a9b316635
search_terms = ["we need to enforce our borders", "learn to speak english",
                "are terrorists","where are you really from", "are spreading virus",
                "i’m not like other", "illegal alien", "illegal immigrant",
                    "illegal immigration", "send her back", "send them back", "illegal criminal"]

In [36]:
def xenophobic_match(tweet):
    """
    Check that tweet has one of the search queries used.
    """
    for s in search_terms:
        if s in tweet.lower():
            return True
    return False

In [37]:
df1['text']=df1['text'].astype(str)
df1["target"]=df1.apply(lambda x : xenophobic_match(x.text), axis=1)
df1['target'].value_counts()


target
False    10000
Name: count, dtype: int64

In [38]:
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import re


In [40]:
nrows = 1000000
df = pd.read_csv('/content/sample_data/Xenophobia.csv', nrows=nrows, encoding='latin1')

# Drop columns not used for modelling
cols_to_drop = ['status_id', 'created_at', 'location']
df.drop(cols_to_drop, axis=1, inplace=True)

# Convert text to string type
df['text'] = df['text'].astype(str)

print("Total number of samples:", len(df))

df.head()

Total number of samples: 554096


,text,favorite_count,retweet_count,followers_count,friends_count,statuses_count
0,wash yo hands\nstay updated\nwash em again\nea...,0.0,0.0,493.0,225.0,12262.0
1,The President inside the with its leadership...,0.0,0.0,10450.0,3722.0,15102.0
2,Itâs hard to focus on anything else with the...,0.0,0.0,6113.0,679.0,3932.0
3,"I mean, seriously. The Acting Deputy Secretary...",0.0,0.0,6384.0,1593.0,189120.0
4,"""""Cuccinelli defended the decision, saying the...",0.0,0.0,6384.0,1593.0,189120.0


In [41]:
# Print a random tweet as a sample

sample_index = 25
print(df.iloc[sample_index])

text               10 Pcs Disposable Activated Carbon Filter for ...
favorite_count                                                   0.0
retweet_count                                                    0.0
followers_count                                               5796.0
friends_count                                                 6238.0
statuses_count                                               11178.0
Name: 25, dtype: object


In [42]:
 #Helper function to remove unwanted patterns

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# Remove Twitter handles from the data

df['text'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")


#Remove punctuations, numbers, and special characters

df['text'] = df['text'].str.replace("[^a-zA-Z#]", " ")

# Remove all words below 3 characters

df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


In [43]:
# Tokenize the tweets

tokenized_tweet = df['text'].apply(lambda x: x.split())
tokenized_tweet

0         [wash, hands, stay, updated, wash, again, heal...
1         [President, inside, with, leadership, learn, m...
2         [Itâs, hard, focus, anything, else, with, up...
3         [mean,, seriously., Acting, Deputy, Secretary,...
4         [""Cuccinelli, defended, decision,, saying, th...
                                ...                        
554091    [Hamilton, County, Health, Department, confirm...
554092    [Iâm, really, excited, about, innovations, t...
554093    [Here, some, helpful, resources, from, regardi...
554094    [have, postponed, ""One, Lean, Food, &amp;, Fa...
554095    [...., Enough, Enough,, ungrateful, greedy, hu...
Name: text, Length: 554096, dtype: object

In [44]:
# Stem the tweets

from nltk.stem.porter import *
stemmer = PorterStemmer()
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x])
tokenized_tweet

0           [wash, hand, stay, updat, wash, again, healthi]
1         [presid, insid, with, leadership, learn, more,...
2         [itâ, hard, focu, anyth, els, with, updat, t...
3         [mean,, seriously., act, deputi, secretari, ho...
4         [""cuccinelli, defend, decision,, say, there, ...
                                ...                        
554091    [hamilton, counti, health, depart, confirm, fi...
554092    [iâm, realli, excit, about, innov, that, go,...
554093    [here, some, help, resourc, from, regard, smal...
554094    [have, postpon, ""one, lean, food, &amp;, farm...
554095    [...., enough, enough,, ungrat, greedi, human,...
Name: text, Length: 554096, dtype: object

In [45]:
# Put tokenized tweets back in dataframe

for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
df['text'] = tokenized_tweet
tokenized_tweet


0                   wash hand stay updat wash again healthi
1         presid insid with leadership learn more about ...
2         itâ hard focu anyth els with updat today mon...
3         mean, seriously. act deputi secretari homeland...
4         ""cuccinelli defend decision, say there enough...
                                ...                        
554091    hamilton counti health depart confirm first ch...
554092    iâm realli excit about innov that go come th...
554093    here some help resourc from regard small busi ...
554094    have postpon ""one lean food &amp; farm"" trai...
554095    .... enough enough, ungrat greedi human age re...
Name: text, Length: 554096, dtype: object